## Localization of RoI using Image Fusion

Import required libraries

In [ ]:
import cv2 as cv
import numpy as np
import pywt 
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity
from skimage.measure import shannon_entropy
import flirimageextractor
from math import sqrt

## Spatial Domain Techniques for Image Fusion

    Spatial techniques directly acting on the image pixels

Weighted Summation Fusion

In [ ]:
# alpha represents weight of first image
# set alpha = 0.5 for unweighted average

def weightedFusion(image1, image2, alpha):
    
    # simple weighted average of images
    
    dst = cv.addWeighted(image1, alpha, image2, 1 - alpha, 0.0)
    
    return dst

PCA Fusion ( Principal Component Analysis )

In [ ]:
# finding weights by feature extraction using PCA 

def pcaFusion(image1, image2):
    
    sz = image1.shape

    data = np.zeros((sz[0]*sz[1]*sz[2], 2))
    data[:, 0] = image1.flatten()
    data[:, 1] = image2.flatten()

    # calculating appropriate weights for images using eigenvectors

    _, eigenvectors, eigenvalues = cv.PCACompute2(data, mean=None)
    if(eigenvalues[0][0] > eigenvalues[1][0]):
        eg = 0
    else:
        eg = 1
        
    ref = abs(eigenvectors[eg])
    s = np.sum(ref)
    
    ref = [ref[0]/s, ref[1]/s]
    dst = (ref[0] * image1 + ref[1] * image2)

    return dst

HSI Fusion ( Hue, Saturation, Intensity )

In [ ]:
# alpha represents weight of first image

def hsvFusion(image1, image2, alpha):

    tmp1 = cv.cvtColor(image1, cv.COLOR_BGR2GRAY, dstCn = 1)
    tmp2 = cv.cvtColor(image2, cv.COLOR_BGR2GRAY, dstCn = 1)

    # converting thermal image to HSV for extraction of color map
    
    hsv_image = cv.cvtColor(image2, cv.COLOR_BGR2HSV)
    hsv_channels = cv.split(hsv_image)

    # adding image intensities (grayscale images) using weights 

    res_channel = cv.addWeighted(tmp1, alpha, tmp2, 1-alpha, 0.0)

    # replacing intensity channel of thermal candidate to weightd channel

    dst = cv.merge([hsv_channels[0], hsv_channels[1], res_channel])
    dst = cv.cvtColor(dst, cv.COLOR_HSV2BGR)

    return dst

## Frequency Domain Techniques for Image Fusion

    Techniques based on signal frequency of pixels (for extraction of features from candidate images)

Laplacian Fusion

In [ ]:
# laplacian pyramid for merging and fusion of images

# generate Gaussian pyramid (pre-emtive step for Lplacian)

def generateGaussian(img, n):

    gD = img.copy()
    gPimg = [gD]
    for i in range(n):
        gD = cv.pyrDown(gD)
        gPimg.append(gD)
    
    return gPimg

# generate Laplacian pyramid from gaussian inputs

def generateLaplace(gPimg, n):
    
    lPimg = [gPimg[n]]
    for i in range(n, 0, -1):
        r,c,ch = gPimg[i-1].shape

        gU = cv.pyrUp(gPimg[i])
        gU = gU[:r, :c]
        
        lP = cv.subtract(gPimg[i-1], gU)
        lPimg.append(lP)
    
    return lPimg

# function to fuse laplace outputs layer-wise while going up to get desired results

def weightedAvg(lPA, lPB, alpha):
    
    LS = []
    
    # weighted averaging for each pyramid layer 
    
    for la, lb in zip(lPA, lPB):
        ls = cv.addWeighted(la, alpha, lb, 1-alpha, 0.0)
        LS.append(ls)
    
    res = LS[0]
    for i in range(1, len(lPA)):
        r,c,ch = LS[i].shape

        res = cv.pyrUp(res)
        res = res[:r, :c]

        res = cv.add(res, LS[i])

    return res

# calling laplace fusion here

def laplaceFusion(image1, image2, alpha, n):
    
    gPA = generateGaussian(image1, n)
    gPB = generateGaussian(image2, n)

    lPA = generateLaplace(gPA, n-1)
    lPB = generateLaplace(gPB, n-1)

    dst = weightedAvg(lPA, lPB, alpha)

    return dst

DCT Fusion ( Discrete Cosine Transforms )

In [ ]:
# discrete Cosine/Fourier tranform for fusion of images

def dctFusion(image1, image2, alpha):

    # channel-wie cosine transforms for input images
    
    b1, g1, r1 = cv.split(image1)
    tmpB1, tmpG1, tmpR1 = cv.dct(np.float32(b1)), cv.dct(np.float32(g1)), cv.dct(np.float32(r1))

    b2, g2, r2 = cv.split(image2)
    tmpB2, tmpG2, tmpR2 = cv.dct(np.float32(b2)), cv.dct(np.float32(g2)), cv.dct(np.float32(r2))
    
    # inverse cosine transforms to regenerate image after weighted sum

    B = cv.idct(cv.addWeighted(tmpB1, alpha, tmpB2, 1-alpha, 0.0))
    G = cv.idct(cv.addWeighted(tmpG1, alpha, tmpG2, 1-alpha, 0.0))
    R = cv.idct(cv.addWeighted(tmpR1, alpha, tmpR2, 1-alpha, 0.0))
    
    dst = cv.merge((B, G, R))

    return dst

DWT Fusion ( Discrete Wavelet Transforms )

In [ ]:
# Discrete Wavelet tranform for fusion (without masking)

def preDWT(image):
    
    b, g, r = cv.split(image)

    # calculating dwt coefficients for each channel

    coeffB, coeffG, coeffR = pywt.dwt2(b, 'bior1.3'), pywt.dwt2(g, 'bior1.3'), pywt.dwt2(r, 'bior1.3')
    return [coeffB, coeffG, coeffR]

def dwtFusion(image1, image2, alpha):
    
    arr1 = preDWT(image1)
    arr2 = preDWT(image2)
    
    arr = []
    for coeff1, coeff2 in zip(arr1, arr2):

        # weighted sum of approximations of images followed by maxima technique for DWT coefficients

        cA = cv.addWeighted(coeff1[0], alpha, coeff2[0], 1-alpha, 0.0)
        cD = []
        for cD1, cD2 in zip(coeff1[1], coeff2[1]):
            cD.append(np.maximum(cD1, cD2))
        arr.append([cA, cD])
    
    # inverse wavelet transforms to regenerate image after weighted sum
    res = []
    for coeff in arr:
        d = pywt.idwt2(coeff, 'haar')
        res.append(d)
    
    dst = cv.merge(res)
    return dst

## Region Segmentation

    Segmentation functions to segment region based on thresholding outputs

Adaptive Thresholding

In [ ]:
def adaptThreshold(image):

    # Adaptive thresholding to generate binary output

    img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    th = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 13, 2)
    th = cv.bitwise_not(th)

    # finding contours and returning coordinates of bounding rectangle

    contours, _ = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv.contourArea)
    x,y,w,h = cv.boundingRect(cnt)
    
    ## to show bounding rectangle of obtained contour on input image, uncomment following lines

    # copy = image.copy()
    # t = cv.rectangle(copy, (x,y), (x+w, y+h), (0,255,0), 1)

    # cv.imshow('adaptive-contour-bound', t)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return x,y,w,h

Otsu Thresholding

In [ ]:
def otsuThreshold(image):

    # Otsu thresholding to generate binary output

    img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    _, th = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    # finding contours and returning coordinates of bounding rectangle

    contours, _ = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv.contourArea)
    x,y,w,h = cv.boundingRect(cnt)

    ## ! to show bounding rectangle of obtained contour on input image, uncomment following lines 

    # copy = image.copy()
    # t = cv.rectangle(copy, (x,y), (x+w, y+h), (0,255,0), 1)

    # cv.imshow('otsu-contour-bound', t)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return x,y,w,h 

Inverted Otsu Thresholding

In [ ]:
def invertOtsuThreshold(image):
    
    # inverted Otsu thresholding (black -> white, white -> black)
    # inverse of otsuThreshold()
    
    img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    _, th = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    th = cv.bitwise_not(th)

    contours, _ = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv.contourArea)
    x,y,w,h = cv.boundingRect(cnt)

    ## ! to show bounding rectangle of obtained contour on input image, uncomment following lines 

    # copy = image.copy()
    # t = cv.rectangle(copy, (x,y), (x+w, y+h), (0,255,0), 1)

    # cv.imshow('inverted_otsu-contour-bound', t)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return x,y,w,h

## Edge-based Segmentation

   Manually tuned canny thresholds for edge detection and segmentation 

Canny edge detection for fused image

In [ ]:
def canny(image, thres1, thres2):

    # finding canny edge

    edges = cv.Canny(image, thres1, thres2)

    # finding contours and returning coordinates of bounding rectangle

    contours, _ = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv.contourArea)
    
    x,y,w,h = cv.boundingRect(cnt)

    ## ! to show bounding rectangle of obtained contour on input image, uncomment following lines

    # copy = image.copy()
    # t = cv.rectangle(copy, (x,y), (x+w, y+h), (0,255,0), 1)

    # cv.imshow('canny-contour-bound', t)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return x,y,w,h

Canny edge detection for best output from fused / visual image

In [ ]:
def genFocus(image, vis, thres1, thres2):
    
    edges1 = cv.Canny(image, thres1, thres2)
    edges2 = cv.Canny(vis, thres1, thres2)  
    
    contours1, _ = cv.findContours(edges1, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours2, _ = cv.findContours(edges2, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    coord1, coord2 = (0,0,0,0), (0,0,0,0)
    
    if len(contours1) != 0:
        cnt1 = max(contours1, key = cv.contourArea)
        coord1 = cv.boundingRect(cnt1)
    if len(contours2) != 0:
        cnt2 = max(contours2, key = cv.contourArea)
        coord2 = cv.boundingRect(cnt2)

    x,y,w,h = max([coord1, coord2], key = lambda k : k[2]*k[3])

    ## ! to show bounding rectangle of obtained contour on input image, uncomment following lines
    
    # copy = image.copy()
    # t = cv.rectangle(copy, (x, y), (x+w, y+h), (0, 255, 0), 1)  

    # cv.imshow('dst', t)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return x, y, w, h

## Adaptive Canny Segmentation

    Using Otsu thresholds as minimum and maximum canny limits for edge detection

In [ ]:
def CannyWithOtsu(image):

    # Otsu thresholding to find threshold metric

    img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    res, _ = cv.threshold(img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    # minimum threshold : 0.5*OT, maximum thresholds: OT

    edges = cv.Canny(img, res*0.5, res)

    # finding contours and returning coordinates of bounding rectangle

    contours, _ = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key = cv.contourArea)
    x,y,w,h = cv.boundingRect(cnt)
    
    ## ! to show bounding rectangle of obtained contour on input image, uncomment following lines

    # copy = image.copy()
    # t = cv.rectangle(copy, (x,y), (x+w, y+h), (0,255,0), 1)

    # cv.imshow('canny_otsu-contour-bound', t)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return x,y,w,h

## Morphological Segmentation

    Morphological Segmentation using Watershed
    

In [ ]:
def watershed(image):

    # Proprocessing steps to define markers for watershed
    #  thresholding using Otsu
    
    gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    _, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

    # Morphological opening to reduce noise

    kernel = np.ones((3,3), np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations = 2)

    # marking sure foreground and background 

    sure_bg = cv.dilate(opening,kernel,iterations=3)
    
    # distance tranform to find centroids of foreground objects

    dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
    _, sure_fg = cv.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)

    # defining markers on foreground using connected components

    _, markers = cv.connectedComponents(sure_fg)
    markers = markers+1
    markers[unknown==255] = 0

    # using markers for watershed

    markers = cv.watershed(image, markers)

    ## ! to show bounding foreground detected by watershed, uncomment following lines
    
    # copy = image.copy()
    # copy[markers == -1] = [0,255,0]
    
    # cv.imshow('watershed-foreground', copy)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return markers

## Color Segmentation
    
    FCM and K-means algorithm for color-based segmentation

In [ ]:
def fcm(image, C):
    temp = image.copy()
    tmp = temp.reshape((-1, 3)).T

    # using skmeans fcm to get cluster centers and ditribution

    centers, u, _, _, _, _, _ = fuzz.cluster.cmeans(tmp, c=C, m=2, error=0.005, maxiter=1000)

    img_clustered = np.argmax(u, axis=0).astype(int)
    cluster2D = img_clustered.reshape((image.shape[0], image.shape[1]))
    
    # changing distribution in accordance to center labels

    res = np.zeros(image.shape)
    for i in range(cluster2D.shape[0]):
        for j in range(cluster2D.shape[1]):
            k = cluster2D[i][j]
            res[i][j] = centers[k]
    
    ## ! to show color segmented output, uncomment following lines 

    # cv.imshow('color-cluster', res)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return res.astype(np.uint8)

def kmeans(image, K):
    
    temp = image.copy()
    tmp = np.float32(temp.reshape((-1, 3)))
    
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)

    # using opencv kmeans to get cluster labels and ditribution

    _, label, center = cv.kmeans(tmp, K, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)

    center = np.uint8(center)
    res = center[label.flatten()].reshape((temp.shape))

    ## ! to show color segmented output, uncomment following lines 
    
    # cv.imshow('color-cluster', res)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    
    return res

## Utility Functions

    Main functions for Localization of ROI

Color segregation step (for layers)

In [ ]:
def colorSeg(image, base, vals):
    image_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    h, _, _ = cv.split(image_hsv)

    bins = np.bincount(h.flatten())
    peaks = np.where(bins > 0)[0]
    
    ## ! to see the hue values detected in the image, uncomment `peaks`
    # print(peaks)
    
    m,n = vals
    pres = False
    for i in peaks:
        if i>=m and i<=n:
            pres = True
            break
    
    if pres:
        mask = cv.inRange(h, m, n)
    else:
        mask = cv.inRange(h, int(peaks[0]), int(peaks[0]))

    ## ! to show filtered hue range from segmented inputs, uncomment following lines  

    # blob = cv.bitwise_and(image, image, mask=mask)

    # cv.imshow('filtered-hue', blob)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnt = max(contours, key=cv.contourArea)
    x,y,w,h = cv.boundingRect(cnt)
    
    ## ! to show bounding rectangle of obtained contour on input image, uncomment following lines

    # copy = image.copy()
    # t = cv.rectangle(copy, (x,y), (x+w, y+h), (0,255,0), 1)

    # cv.imshow('canny_otsu-contour-bound', t)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return x, y, w, h

Main focus() function to localize RoI

In [ ]:
def focus(image, vis, sz, K, iter, vals, index):

    if(iter == 0):

        ## ! Last step segmentation can be changed here.... 
        ## ! change function below to get coordinates
     
        return CannyWithOtsu(image)
    
    if(sz[1] != 0):
        temp = image[:, sz[0]:sz[1]]
        vis_temp = vis[:, sz[0]:sz[1]]
    else:
        temp = image.copy()
        vis_temp = vis.copy()
        
    res = kmeans(temp, K)
    base_X, base_Y, ext_X, ext_Y = colorSeg(res, sz[0], vals[iter-1])

    # resizing image to localize area obtained from color segmentation 

    t = temp[base_Y:base_Y + ext_Y, base_X:base_X + ext_X]
    v = vis_temp[base_Y:base_Y + ext_Y, base_X:base_X + ext_X].astype(np.uint8)

    # recursive call to focus, depending on number of layers chosen for color segmentation

    x, y, w, h = focus(t, v, [0,0], K, iter-1, vals, index)

    return base_X + x, base_Y + y, w, h

Mask Obtained Localization with black background 

In [ ]:
def maskROI(coord, image):
    
    x1,y1,x2,y2 = coord
    mask = np.zeros(image.shape[:2], dtype="uint8")
    
    cv.rectangle(mask, (x1,y1), (x2,y2), 255, -1)
    masked = cv.bitwise_and(image, image, mask=mask)
    
    ## ! to show masked localized region

    # cv.imshow('res', masked)
    # cv.waitKey(0)
    # cv.destroyAllWindows()

    return masked

## Performance Evaluation

    Evaluation Metrics for localization and fusion

Helper functions

In [ ]:
# intersection of global truth and obtained localization

def intersection(a, b):
    
    x = max(a[0], b[0])
    y = max(a[1], b[1])
    w = min(a[2], b[2]) - x
    h = min(a[3], b[3]) - y
    
    if w<0 or h<0: 
        return 0

    return w*h

# union of global truth and localized region

def union(a, b):
    
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[2], b[2]) - x
    h = max(a[3], b[3]) - y
    
    return w*h

Evaluation metrics to assess fusion techniques

In [ ]:
# peak signal to noise ratio

def PSNR(original, fused):

    original = cv.resize(original, (fused.shape[1], fused.shape[0]))
    
    return peak_signal_noise_ratio(original, fused)

# mean squared error

def MSE(original, fused):

    original = cv.resize(original, (fused.shape[1], fused.shape[0]))
    
    return mean_squared_error(original, fused)

# structural similarity

def SSIM(original, fused):
    
    original = cv.resize(original, (fused.shape[1], fused.shape[0]))

    original = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
    fused = cv.cvtColor(fused, cv.COLOR_BGR2GRAY)

    score, _ = structural_similarity(original, fused, full=True)
    
    return score

Evaluation metrics to assess localization procedure

In [ ]:
# shannon entropy of localized region

def ENT(coord, fused):
    
    x1, y1, x2, y2 = coord
    
    return ('Entropy: ' + str(shannon_entropy(fused[y1:y2, x1:x2])))

# Mean of temperatures (from thermal image) of localized region

def thermal_mean(coord, path):
    
    x1, y1, x2, y2 = coord

    flir = flirimageextractor.FlirImageExtractor()
    flir.process_image(path)

    return ('Mean temperature: ' + str(np.mean(flir.get_thermal_np()[y1:y2, x1:x2])))

# Area of localized region

def area(coord):

    x1,y1,x2,y2 = coord
    
    return ('Area: ' +  str((x2-x1)*(y2-y1)))

# Euclidean distance between centers of localized region and ground truth

def dist(coord, center):
    
    x1, y1, x2, y2 = coord

    obtX, obtY = (x1+x2)/2, (y1+y2)/2
    cenX, cenY = center

    return ('Centroid Distance: ' + str(sqrt((obtX - cenX)**2 + (obtY - cenY)**2)))

# Intersection over union (overlapping region metric) for localized region and ground truth
 
def IOU(coord1, coord2):
    
    return ('mAP: ' + str(intersection(coord1, coord2) / union(coord1, coord2)))

Evaluate function 

In [ ]:
# set evaluation metrics here

def evaluate(image, coord, path, center, index, gt):

    ## ! uncomment to save metrics to file
    
    # file = <set file path> 
    # with open(file, "a+") as f:
    #     f.write('#### ' + str(index) + ' ###\n' + ENT(gt, image) + '\n' + thermal_mean(gt, path) + '\n' + area(gt) + '\n' + dist(coord, center) + '\n' + IOU(coord, gt) + '\n')
    
    print(str(index) + ':: ' + IOU(coord, gt))
    print(str(index) + ':: ' + dist(coord, center))
    print(str(index) + ':: ' + area(coord))
    print(str(index) + ':: ' + thermal_mean(coord, path))
    print(str(index) + ':: ' + ENT(coord, image))

## Image Processing 

    Input images and calling functions

Output

In [ ]:
def ImgProcess(image1, image2, path, center, r, index, gt):

    ########## Fusion Process ############

    ## ! set fusion function here 

    fused = hsvFusion(image1, image2, 0.7)
    fused = cv.resize(fused, (image1.shape[1], image1.shape[0]))

    ## ! save fusion results to folder
    # p = <directory_path> + str(index) + '.jpg'
    
    # cv.imwrite(p, dst)

    ###########   Localization process ###########

    # define parameters here
    # k -> clustering parameter
    # vals -> hue ranges for layers
    # len(vals) = number of layers, .... 3 here

    K = 6
    vals = [ (0, 10), (0, 15), (0, 20) ]

    # calling focus function
    # focus( fused_image,  visual_image,  crop_coordinates_for_x-axis,  K-value_of_sgmentation,  number_of_layers,  hue_ranges_for_layers,  image_index )

    x,y,w,h = focus(fused, image1, r, 6, 3, vals, index)
    x1, y1, x2, y2 = x+r[0], y, x+r[0]+w, y+h
    coords = (x1,y1,x2,y2)
    
    RoI = fused.copy()
    cv.rectangle(RoI, (x1, y1), (x2, y2), (0, 255, 0), 1)

    ## ! save localization results to folder
    # p = <directory_path> + str(index) + '.jpg'
    
    # cv.imwrite(p, copy)

    ########### Results #################

    ## ! show fused image result here

    cv.imshow('Fused image', fused)
    cv.waitKey(0)
    cv.destroyAllWindows()

    ## ! show localized region result here

    cv.imshow('Localized region', RoI)
    cv.waitKey(0)
    cv.destroyAllWindows()

    fused = cv.imread('fused.jpg')

    ############# Evaluate results ####################

    # evaluate( fused_image,  RoI_coordinates,  thermal_image_path,  center_of_global_truth,  image_index,  global_truth_coordinates )

    evaluate(fused, coords, path, center, index, gt)
    
    ## ! return fused and RoI images for further computation
    
    # return fused, RoI

def ReadImg(index, centers, crop, GT):

    # set visual and thermal images here (from path)
    Tpath = './Assets/fusion/Thermal/' + str(index)+ '.jpg'
    Vpath = './Assets/fusion/Visual/' + str(index)+ '.jpg'

    img = cv.imread(Vpath)
    Timg = cv.imread(Tpath)

    # ImgProcess( visual_image,  thermal_image,  thermal_image_path,  image_center,  image_crop,  image_index,  global_truth )

    return ImgProcess(img, Timg, Tpath, centers[index-1], crop[index-1], index, GT[index-1])

Manually calculated Constants

In [ ]:
# crop coordinates for X-axis (for input images)

crop = [[80,240], [80,240], [80,240], [80,240], [0, 150], [80,240], [80,240], [100,250], [80,240], [100,260], [80,240], [100,250]]

# global truth coordinates
# (0,0,1,1) for images with irregular global truths

centers = [(162, 65), (190, 60), (200, 60), (142, 124), (87, 63), (197, 52), (137, 74), (215, 70), (0, 0), (209, 86), (160, 62), (212, 153)]
GT = [(148,54,172,75), (0,0,1,1), (190,40,213,62), (127, 114,153,140), (67,48,92,80), (180,41,208,74), (0,0,1,1), (207,67,222,85), (0,0,1,1), (195,68,220,100), (148,52,172,73), (207,139,218,161)]

Caller function

In [ ]:
# -- images are stored as str(index).jpg ..... change in ReadImg for different custom path
## ! for looped results based on index, uncomment

# for i in range(1, len(GT)+1):
    # ReadImg(1+i, centers, crop, GT)

# Single image result (index is added manually -- corresponding to calculated constants)

# Imp note .... Desirable results may not be obtained in the first or second run ..... try running 3-4 times.
# if output remains unchanged, it implies algorithm fails

ReadImg(12, centers, crop, GT)